In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [3]:
booksDF = pd.read_csv('datasets/books-crossing/BX-Books.csv', sep=';', encoding='latin-1',  error_bad_lines=False)
usersDF = pd.read_csv('datasets/books-crossing/BX-Users.csv', sep=';', encoding="latin-1", error_bad_lines=False)
ratingsDF = pd.read_csv('datasets/books-crossing/BX-Book-Ratings.csv', sep=';', encoding="latin-1", error_bad_lines=False)

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/Users/mac/opt/anaconda3/envs/recoxplainer/lib/python3.6/site-packages/IPy

In [4]:
booksDF

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...
271355,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [5]:
booksDF.insert(0, 'id', range(1, 1 + len(booksDF)))
booksDF

,id,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,1,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,3,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,5,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...,...
271355,271356,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,271357,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,271358,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,271359,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [6]:
booksDF.rename(columns = {'ISBN':'book_id', 'Book-Title':'original_title', 'Book-Author':'authors'}, inplace=True)

In [7]:
booksDF

,id,book_id,original_title,authors,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,1,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,3,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,5,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...
...,...,...,...,...,...,...,...,...,...
271355,271356,0440400988,There's a Bat in Bunk Five,Paula Danziger,1988,Random House Childrens Pub (Mm),http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...,http://images.amazon.com/images/P/0440400988.0...
271356,271357,0525447644,From One to One Hundred,Teri Sloat,1991,Dutton Books,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...,http://images.amazon.com/images/P/0525447644.0...
271357,271358,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker,2004,HarperSanFrancisco,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...,http://images.amazon.com/images/P/006008667X.0...
271358,271359,0192126040,Republic (World's Classics),Plato,1996,Oxford University Press,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...,http://images.amazon.com/images/P/0192126040.0...


In [8]:
booksDF = booksDF.drop('Year-Of-Publication',1).drop('Publisher', 1).drop('Image-URL-S', 1).drop('Image-URL-M',1).drop('Image-URL-L',1)

In [9]:
booksDF

,id,book_id,original_title,authors
0,1,0195153448,Classical Mythology,Mark P. O. Morford
1,2,0002005018,Clara Callan,Richard Bruce Wright
2,3,0060973129,Decision in Normandy,Carlo D'Este
3,4,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,5,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...,...
271355,271356,0440400988,There's a Bat in Bunk Five,Paula Danziger
271356,271357,0525447644,From One to One Hundred,Teri Sloat
271357,271358,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271358,271359,0192126040,Republic (World's Classics),Plato


In [10]:
ratingsDF

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [11]:
ratingsDF.rename(columns = {'ISBN':'book_id'}, inplace=True)
ratingsDF

,User-ID,book_id,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6
...,...,...,...
1149775,276704,1563526298,9
1149776,276706,0679447156,0
1149777,276709,0515107662,10
1149778,276721,0590442449,10


In [12]:
ratingsDF = ratingsDF.merge(booksDF, on='book_id')
ratingsDF

,User-ID,book_id,Book-Rating,id,original_title,authors
0,276725,034545104X,0,2967,Flesh Tones: A Novel,M. J. Rose
1,2313,034545104X,5,2967,Flesh Tones: A Novel,M. J. Rose
2,6543,034545104X,0,2967,Flesh Tones: A Novel,M. J. Rose
3,8680,034545104X,5,2967,Flesh Tones: A Novel,M. J. Rose
4,10314,034545104X,9,2967,Flesh Tones: A Novel,M. J. Rose
...,...,...,...,...,...,...
1031131,276688,0517145553,0,69528,Mostly Harmless,Douglas Adams
1031132,276688,1575660792,7,69497,Gray Matter,Shirley Kennett
1031133,276690,0590907301,0,69539,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey
1031134,276704,0679752714,0,69546,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman


In [13]:
ratingsDF = ratingsDF.drop('book_id',1).drop('original_title',1).drop('authors',1)

In [14]:
ratingsDF.rename(columns = {'User-ID':'user_id', 'Book-Rating':'rating','id':'book_id'}, inplace=True)
ratingsDF

,user_id,rating,book_id
0,276725,0,2967
1,2313,5,2967
2,6543,0,2967
3,8680,5,2967
4,10314,9,2967
...,...,...,...
1031131,276688,0,69528
1031132,276688,7,69497
1031133,276690,0,69539
1031134,276704,0,69546


In [15]:
ratingsDF = ratingsDF.drop(ratingsDF.index[59:78])

In [16]:
ratingsDF

,user_id,rating,book_id
0,276725,0,2967
1,2313,5,2967
2,6543,0,2967
3,8680,5,2967
4,10314,9,2967
...,...,...,...
1031131,276688,0,69528
1031132,276688,7,69497
1031133,276690,0,69539
1031134,276704,0,69546


In [17]:
ratingsDF = ratingsDF.drop(ratingsDF.index[874:976])
ratingsDF

,user_id,rating,book_id
0,276725,0,2967
1,2313,5,2967
2,6543,0,2967
3,8680,5,2967
4,10314,9,2967
...,...,...,...
1031131,276688,0,69528
1031132,276688,7,69497
1031133,276690,0,69539
1031134,276704,0,69546


In [18]:
ratingsDF = ratingsDF.drop(ratingsDF.index[111111:111434])
ratingsDF.shape

(1030692, 3)

In [19]:
ratingsDF = ratingsDF.drop(ratingsDF.index[873547:873747])
ratingsDF.shape

(1030492, 3)

In [20]:
ratingsDF = ratingsDF.drop(ratingsDF.index[988883:988983])
ratingsDF.shape

(1030392, 3)

In [21]:
ratingsDF = ratingsDF.drop(ratingsDF.index[988883:988983])
ratingsDF.shape

(1030292, 3)

In [22]:
ratingsDF = ratingsDF.drop(ratingsDF.index[167:367])
ratingsDF.shape

(1030092, 3)

In [23]:
ratingsDF = ratingsDF.drop(ratingsDF.index[98274:98400])
ratingsDF.shape

(1029966, 3)

In [24]:
ratingsDF = ratingsDF.drop(ratingsDF.index[567:679])
ratingsDF.shape

(1029854, 3)

In [25]:
ratingsDF = ratingsDF.drop(ratingsDF.index[10:50])
ratingsDF.shape

(1029814, 3)

In [26]:
ratingsDF = ratingsDF.drop(ratingsDF.index[89:95])
ratingsDF.shape

(1029808, 3)

In [27]:
ratingsDF = ratingsDF.drop(ratingsDF.index[2837:2849])
ratingsDF.shape

(1029796, 3)

In [28]:
ratingsDF = ratingsDF.drop(ratingsDF.index[1098:1111])
ratingsDF.shape

(1029783, 3)

In [29]:
ratingsDF = ratingsDF.drop(ratingsDF.index[3847:3869])
ratingsDF.shape

(1029761, 3)

In [30]:
ratingsDF = ratingsDF.drop(ratingsDF.index[347:358])
ratingsDF.shape

(1029750, 3)

In [31]:
ratingsDF = ratingsDF.drop(ratingsDF.index[945:956])
ratingsDF.shape

(1029739, 3)

In [32]:
ratingsDF = ratingsDF.drop(ratingsDF.index[756:823])
ratingsDF.shape

(1029672, 3)

In [33]:
ratingsDF = ratingsDF.drop(ratingsDF.index[23:56])
ratingsDF.shape

(1029639, 3)

In [34]:
ratingsDF = ratingsDF.drop(ratingsDF.index[84:97])
ratingsDF.shape

(1029626, 3)

In [35]:
ratingsDF = ratingsDF.drop(ratingsDF.index[57:78])
ratingsDF.shape

(1029605, 3)

In [36]:
ratingsDF = ratingsDF.drop(ratingsDF.index[123:234])
ratingsDF.shape

(1029494, 3)

In [37]:
ratingsDF = ratingsDF.drop(ratingsDF.index[2543:2754])
ratingsDF.shape

(1029283, 3)

In [38]:
ratingsDF = ratingsDF.drop(ratingsDF.index[37475:37999])
ratingsDF.shape

(1028759, 3)

In [39]:
ratingsDF = ratingsDF.drop(ratingsDF.index[10000:20000])
ratingsDF.shape

(1018759, 3)

In [40]:
ratingsDF = ratingsDF.drop(ratingsDF.index[8888:9999])
ratingsDF.shape

(1017648, 3)

In [41]:
ratingsDF = ratingsDF.drop(ratingsDF.index[6666:7777])
ratingsDF.shape

(1016537, 3)

In [42]:
ratingsDF = ratingsDF.drop(ratingsDF.index[3333:4444])
ratingsDF.shape

(1015426, 3)

In [43]:
ratingsDF = ratingsDF.drop(ratingsDF.index[4444:5555])
ratingsDF.shape

(1014315, 3)

In [44]:
ratingsDF = ratingsDF.drop(ratingsDF.index[9876:10987])
ratingsDF.shape

(1013204, 3)

In [45]:
ratingsDF = ratingsDF.drop(ratingsDF.index[12345:54321])
ratingsDF.shape

(971228, 3)

In [46]:
ratingsDF = ratingsDF.drop(ratingsDF.index[8754:9999])
ratingsDF.shape

(969983, 3)

In [47]:
ratingsDF = ratingsDF.drop(ratingsDF.index[12999:2999])
ratingsDF.shape

(969983, 3)

In [48]:
ratingsDF = ratingsDF.drop(ratingsDF.index[823:6552])
ratingsDF.shape

(964254, 3)

In [49]:
ratingsDF = ratingsDF.drop(ratingsDF.index[162366:23746])
ratingsDF.shape

(964254, 3)

In [50]:
ratingsDF = ratingsDF.drop(ratingsDF.index[37777:47777])
ratingsDF.shape

(954254, 3)

In [51]:
ratingsDF = ratingsDF.drop(ratingsDF.index[9999:19999])
ratingsDF.shape

(944254, 3)

In [52]:
ratingsDF = ratingsDF.drop(ratingsDF.index[88822:90000])
ratingsDF.shape

(943076, 3)

In [53]:
ratingsDF = ratingsDF.drop(ratingsDF.index[28377:3999])
ratingsDF.shape

(943076, 3)

In [54]:
ratingsDF = ratingsDF.drop(ratingsDF.index[40000:50000])
ratingsDF.shape

(933076, 3)

In [55]:
ratingsDF = ratingsDF.drop(ratingsDF.index[40000:70000])
ratingsDF.shape

(903076, 3)

In [56]:
ratingsDF = ratingsDF.drop(ratingsDF.index[30000:60000])
ratingsDF.shape

(873076, 3)

In [57]:
ratingsDF = ratingsDF.drop(ratingsDF.index[20000:50000])
ratingsDF.shape

(843076, 3)

In [58]:
ratingsDF = ratingsDF.drop(ratingsDF.index[80000:85000])
ratingsDF.shape

(838076, 3)

In [59]:
ratingsDF = ratingsDF.drop(ratingsDF.index[10000:50000])
ratingsDF.shape

(798076, 3)

In [60]:
ratingsDF = ratingsDF.drop(ratingsDF.index[40000:50000])
ratingsDF.shape

(788076, 3)

In [61]:
ratingsDF = ratingsDF.drop(ratingsDF.index[60000:75000])
ratingsDF.shape

(773076, 3)

In [62]:
ratingsDF = ratingsDF.drop(ratingsDF.index[6666:7666])
ratingsDF.shape

(772076, 3)

In [63]:
ratingsDF = ratingsDF.drop(ratingsDF.index[66666:76666])
ratingsDF.shape

(762076, 3)

In [64]:
ratingsDF = ratingsDF.drop(ratingsDF.index[100000:200000])
ratingsDF.shape

(662076, 3)

In [65]:
ratingsDF = ratingsDF.drop(ratingsDF.index[400000:80000])
ratingsDF.shape

(662076, 3)

In [66]:
ratingsDF = ratingsDF.drop(ratingsDF.index[12345:23456])
ratingsDF.shape

(650965, 3)

In [67]:
ratingsDF = ratingsDF.drop(ratingsDF.index[8765:9999])
ratingsDF.shape

(649731, 3)

In [68]:
ratingsDF = ratingsDF.drop(ratingsDF.index[100000:300000])
ratingsDF.shape

(449731, 3)

In [69]:
ratingsDF = ratingsDF.drop(ratingsDF.index[200000:500000])
ratingsDF.shape

(200000, 3)

# Start of Copying code

In [70]:
listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0;
testdf = ratingsDF
testdf=testdf[['user_id','rating']].groupby(testdf['book_id'])
for groupKey in testdf.groups.keys():
    tempDict={}

    groupDF = testdf.get_group(groupKey)
    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]]=groupDF.iloc[i,1]
    indexMap[ptr]=groupKey
    reverseIndexMap[groupKey] = ptr
    ptr=ptr+1
    listOfDictonaries.append(tempDict)

In [71]:
from sklearn.feature_extraction import DictVectorizer
dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)

In [72]:
from sklearn.metrics.pairwise import cosine_similarity
pairwiseSimilarity = cosine_similarity(vector)

In [77]:
def printBookDetails(bookID):
    print("Title:", bookDF[bookDF['id']==bookID]['original_title'].values[0])
    print("Author:",bookDF[bookDF['id']==bookID]['authors'].values[0])
    print("Printing Book-ID:",bookID)
    print("=================++++++++++++++=========================")


def getTopRecommandations(bookID):
    row = reverseIndexMap[bookID]
    #print("------INPUT BOOK--------")
    #printBookDetails(bookID)
    #print("-------RECOMMENDATIONS----------")
    #similarBookIDs = [printBookDetails(indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-12:-2][::-1]]
    return np.argsort(pairwiseSimilarity[row])[-12:-2][::-1]

# Getting Base Book for each User

In [78]:
len(ratingsDF['user_id'].unique())

34083

In [79]:
#getting base book for all users
booksUsedForRecommendations = []
usersList = ratingsDF.groupby(by='user_id')
userIds = ratingsDF['user_id'].unique()

for x in range(len(userIds)):
    allUserRatings = pd.DataFrame(usersList.get_group(userIds[x]))
    allUserRatings = allUserRatings.sort_values(['rating'], ascending=[False])
    itemUsedForRecommendation = allUserRatings.head(1)
    booksUsedForRecommendations.append(itemUsedForRecommendation)

# Recommending 10 books for each user based on base book



In [80]:
ids = []
recommendations = []
titles = []
for x in range(len(userIds)):
    userId = userIds[x]
    bookId = booksUsedForRecommendations[x].book_id.item()
    result = getTopRecommandations(bookId) #array of book ids
    for i in range (len(result)):
        ids.append(userIds[x])
        recommendations.append(result[i] + 1)
        bookTitle = booksDF[booksDF['id']==(result[i]+1)]['original_title'].values[0]
        titles.append(bookTitle)
allUsersRecommendations_df = pd.DataFrame(list(zip(ids, recommendations,titles)),
            columns =['user_id', 'book_id', 'title'])

In [81]:
allUsersRecommendations_df

,user_id,book_id,title
0,276725,1675,Glad Rags
1,276725,406,The Perfect Storm : A True Story of Men Agains...
2,276725,418,Ohitika Woman
3,276725,415,The Robber Bride
4,276725,411,Hannibal
...,...,...,...
340825,60937,13714,Hellspark
340826,60937,13713,You Can Heal Your Life/101
340827,60937,13712,"River, Cross My Heart : A Novel"
340828,60937,13711,"It Wasn't Always Easy, but I Sure Had Fun: (Th..."


# Explaning using KNN

# Getting Similar Items

In [73]:
knn_items_dict = {}

In [74]:
pairwiseSimilarity.shape

(41144, 41144)

In [75]:
min_val = pairwiseSimilarity.min() - 1

In [76]:
for i in range(41144):
            pairwiseSimilarity[i, i] = min_val
            knn_to_item_i = (-pairwiseSimilarity[i, :]).argsort()[:10]
            knn_items_dict[i] = knn_to_item_i

# Calculating intersection between ratings and similar items

In [82]:
pointer = 0
explanations = []
for x in range (len(userIds)):
    user_ratings = usersList.get_group(userIds[x]).book_id.values #give it userID
    counter = 0
    while counter < 10:
        #print(pointer)
        recBookId = recommendations[pointer]
        #print(recBookId)
        recBookTitle = titles[pointer]
        #print(recommendedMovie)
        #rec_Origin_id = int(recommendedMovie.origin_iid)
        #print(rec_item_id)
        sim_items = knn_items_dict[recBookId - 1]
        explanation =  set(sim_items) & set(user_ratings)
        explanations.append(explanation)
        pointer = pointer + 1
        counter = counter + 1
allUsersRecommendations_df['explanations'] = explanations

In [83]:
allUsersRecommendations_df

,user_id,book_id,title,explanations
0,276725,1675,Glad Rags,{}
1,276725,406,The Perfect Storm : A True Story of Men Agains...,{}
2,276725,418,Ohitika Woman,{}
3,276725,415,The Robber Bride,{}
4,276725,411,Hannibal,{}
...,...,...,...,...
340825,60937,13714,Hellspark,{}
340826,60937,13713,You Can Heal Your Life/101,{}
340827,60937,13712,"River, Cross My Heart : A Novel",{}
340828,60937,13711,"It Wasn't Always Easy, but I Sure Had Fun: (Th...",{}


# Model Fidelity

In [84]:
expl = allUsersRecommendations_df[[len(x) > 0 for x in allUsersRecommendations_df.explanations]]
fidelity = expl.groupby('user_id')['title'].count() / 10
modelFidelity = sum(fidelity)/len(userIds)

In [85]:
modelFidelity

0.0006894932957779548

# AR

In [86]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [87]:
#Computing Assosciation Rules

rules = None
item_sets = [
        [item for item in ratingsDF[ratingsDF.user_id == user].book_id]
        for user in ratingsDF.user_id.unique()
    ]

te = TransactionEncoder()
te_ary = te.fit(item_sets).transform(item_sets)

df = pd.DataFrame(te_ary, columns=te.columns_)

frequent_itemsets = apriori(df,
                            min_support=.1,
                            use_colnames=True,
                            max_len=2)

rules = association_rules(frequent_itemsets,
                            metric="lift",
                            min_threshold=.1)
rules = rules[(rules['confidence'] > .1) &
               (rules['lift'] > .1)]

rules.consequents = [list(row.consequents)[0] for _, row in rules.iterrows()]
rules.antecedents = [list(row.antecedents)[0] for _, row in rules.iterrows()]

rules = rules[["consequents", "antecedents", "confidence"]]

ValueError: cannot call `vectorize` on size 0 inputs unless `otypes` is set